In [1]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql.functions import lower, col
from pyspark.sql.functions import regexp_extract
from pyspark.sql.window import Window
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('q5').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()
#For VIRTUAL
#spark = SparkSession.builder.master('spark://picachu-VirtualBox:7077').appName('q5').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()

#For LIVY
#spark=SparkSession.builder.master('spark://localhost:7077').appName("Testing").getOrCreate()
#and changed the filepaths to '/home/administrator/Downloads/movielens/file_name.csv'
stagemetrics = StageMetrics(spark)
stagemetrics.begin()#start measuring performance

df_tag= (spark.read
     .format("csv")
     .option('header', 'true')
     .option('delimiter', ",")
     .option("inferSchema","true")
     .load("tag.csv") 
    )

lc=df_tag.select('movieId',lower(col('tag')).alias('tag_lc')) #lower case tags

df_movie= (spark.read
     .format("csv")
     .option('header', 'true')
     .option('delimiter', ",")
     .option("inferSchema","true")
     .load("movie.csv") 
    )

withyear=df_movie.select('movieId','title',regexp_extract('title', r'(\d{4})', 0).alias('year')) #get year of each movie & adding it to a new column

year2015=withyear.filter(withyear['year']==2015) #for only year 2015
#joining titles and tags for 2015 # no duplicates
joined=lc.join(year2015, year2015["movieId"]==lc["movieId"], "inner").hint("broadcast").select('title','tag_lc').distinct() 
#joined.orderBy('title').show(18) #requested answer for query 5 # the first 5 movie titles with tags


windowSpec= Window.orderBy('title')
temp=joined.select('title').distinct()
temp=temp.withColumn('row',row_number().over(windowSpec))

top5=temp.join(joined, joined['title']==temp['title'], "inner").select(temp['title'],'tag_lc','row').distinct()
top5.select('title','tag_lc').filter(top5['row']<=5).orderBy('title').show()  


stagemetrics.end()#stop measuring performance
stagemetrics.print_report()#print performance metrics

spark.stop() # stop spark

+--------------------+--------------------+
|               title|              tag_lc|
+--------------------+--------------------+
|A Grain of Truth ...|   abel korzeniowski|
|A Grain of Truth ...|                dark|
|A Grain of Truth ...|       borys lankosz|
|A Grain of Truth ...|         do zassania|
|A Grain of Truth ...|       investigation|
|A Grain of Truth ...|               crime|
|A Grain of Truth ...|         jerzy trela|
|A Grain of Truth ...|  robert wieckiewicz|
|A Grain of Truth ...|polish-jewish his...|
|A Grain of Truth ...|   social commentary|
|A Grain of Truth ...|              poland|
|A Grain of Truth ...|          sandomierz|
|A Grain of Truth ...|            thriller|
|A Walk in the Woo...|          ken kwapis|
| Advantageous (2015)|      jennifer phang|
|As We Were Dreami...|     based on a book|
|Average Italian (...|               drugs|
|Average Italian (...|    marcello macchia|
+--------------------+--------------------+


Scheduling mode = FIFO
Spark C